In [1]:
import pandas as pd
import tqdm
import glob
import os

In [2]:
files = glob.glob("../data/wikipedia/*.parquet")
files = [f for f in files if "all" not in f]

In [3]:
chunk_size = 3_000_000

In [4]:
def output_seperated_files(token_length, stride, drop_category):    
    output_dir = f"../data/wikipedia/sep_chunk/token_length{token_length}_stride_sentence{stride}_drop_category{drop_category}"
    os.makedirs(output_dir, exist_ok=True)
    chunk = 1
    df_wiki_sep = []
    for file in files:
        df_wiki = pd.read_parquet(file)
        df_wiki = df_wiki.drop("categories", axis=1)
        for i in tqdm.tqdm(range(len(df_wiki))):
            series = df_wiki.iloc[i]
            text = series["text"]
            if drop_category:
                idx = text.find(" Category:")
                text = text[:idx]

            text_space = text.split(".")
            text = ""
            text_stride = []

            series_ = series.to_dict()
            series_["text"] = " ".join(text_space[:stride*2])
            series_["title_only"] = True
            df_wiki_sep.append(series_)
            
            appended = False
            for j in range(0, len(text_space)):
                series_ = series.to_dict()
                text += text_space[j] + "."
                text_stride.append(text_space[j])
                text_stride = text_stride[-stride:]
                appended = False
                if len(text.split(" ")) > token_length:
                    series_["text"] = text
                    series_["title_only"] = False
                    df_wiki_sep.append(series_)
                    text = ".".join(text_stride)
                    appended = True
            if not appended:
                series_["text"] = text
                series_["title_only"] = False
                df_wiki_sep.append(series_)

            if len(df_wiki_sep) > chunk_size:
                print(f"chunk {chunk}")
                df_wiki_sep = pd.DataFrame(df_wiki_sep)
                df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
                df_wiki_sep = []
                chunk += 1
    df_wiki_sep = pd.DataFrame(df_wiki_sep)
    df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
    

In [5]:
output_seperated_files(token_length=150, stride=5, drop_category=True)

 76%|██████████████████████████████████████████████               | 334641/442726 [01:54<00:32, 3305.82it/s]

chunk 1


 61%|█████████████████████████████████████████████████████████████████                                          | 198710/326585 [01:14<00:48, 2655.20it/s]

chunk 2


 46%|█████████████████████████████████████████████████▌                                                         | 192264/414595 [01:06<01:43, 2157.64it/s]

...

 78%|███████████████████████████████████████████████████████████████████████████████████▌                       | 287414/367833 [02:13<00:28, 2854.51it/s]

chunk 19


 36%|██████████████████████████████████████▉                                                                     | 62297/172595 [00:31<00:55, 1984.58it/s]

chunk 20


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35525/35525 [00:10<00:00, 3322.46it/s]
